In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
import tsensor
import torchinfo
import tensorboard

import torch
from torch import nn, optim, utils, tensor, from_numpy
from torch.utils.data import Dataset, DataLoader
import lightning as l
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

In [2]:
from typing import Any
from lightning.pytorch.utilities.types import STEP_OUTPUT, OptimizerLRScheduler


class ANN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x
    
class LightningModel(l.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        y = y.view(-1, 1)
        loss = nn.MSELoss()(y_hat, y)
        self.log("train_loss", loss)
        return loss
    
    def test_step(self, batch, batch_idx):
        # this is the test loop
        x, y = batch
        y_hat = self.model(x)
        y = y.view(-1, 1)
        loss = F.mse_loss(y_hat, y)
        self.log("test_loss", loss)
    
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
    def validation_step(self, batch):
        x, y = batch
        y_hat = self.model(x)
        y = y.view(-1, 1)
        loss = nn.MSELoss()(y_hat, y)
        self.log("val_loss", loss)
        return loss

In [3]:
data_df = load_diabetes(as_frame=True).frame.astype(np.float32)

train_df, test_df = train_test_split(data_df, test_size=0.3, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.3, random_state=42)

train_target = from_numpy(train_df.pop('target').values)
val_target = from_numpy(val_df.pop('target').values)
test_target = from_numpy(test_df.pop('target').values)

train_tensor = from_numpy(train_df.values)
test_tensor = from_numpy(test_df.values)
val_tensor = from_numpy(val_df.values)

train_dataset = utils.data.TensorDataset(train_tensor, train_target)
val_dataset = utils.data.TensorDataset(val_tensor, val_target)
test_dataset = utils.data.TensorDataset(test_tensor, test_target)

train_dataloader = DataLoader(train_dataset, batch_size=64)
val_dataloader = DataLoader(val_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

In [4]:
torchinfo.summary(ANN(), (64, 10,))

Layer (type:depth-idx)                   Output Shape              Param #
ANN                                      [64, 1]                   --
├─Linear: 1-1                            [64, 64]                  704
├─ReLU: 1-2                              [64, 64]                  --
├─Dropout: 1-3                           [64, 64]                  --
├─Linear: 1-4                            [64, 128]                 8,320
├─ReLU: 1-5                              [64, 128]                 --
├─Dropout: 1-6                           [64, 128]                 --
├─Linear: 1-7                            [64, 1]                   129
Total params: 9,153
Trainable params: 9,153
Non-trainable params: 0
Total mult-adds (M): 0.59
Input size (MB): 0.00
Forward/backward pass size (MB): 0.10
Params size (MB): 0.04
Estimated Total Size (MB): 0.14

In [5]:
model = LightningModel(ANN())
trainer = l.Trainer(callbacks=[EarlyStopping(monitor="val_loss", mode="min")], max_epochs=100, default_root_dir="checkpoints")

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [6]:
trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)


  | Name  | Type | Params
-------------------------------
0 | model | ANN  | 9.2 K 
-------------------------------
9.2 K     Trainable params
0         Non-trainable params
9.2 K     Total params
0.037     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/richardstrange/opt/miniconda3/envs/ml-portfolio/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/Users/richardstrange/opt/miniconda3/envs/ml-portfolio/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/Users/richardstrange/opt/miniconda3/envs/ml-portfolio/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:293: The number of training batches (4) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training e

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


In [10]:
%reload_ext tensorboard
%tensorboard --logdir=./lightning_logs/

Reusing TensorBoard on port 6007 (pid 22992), started 0:00:13 ago. (Use '!kill 22992' to kill it.)